# Genetic Improvement Demo

### Create the initial population of Genomes

In [1]:
# system imports
import os, sys
from tqdm import tqdm 


# set the project root directory (~/projects/GIMC)
project_root = os.path.expanduser("~/projects/GIMC")

# add project root to sys.path
sys.path.append(project_root)

# project imports
from genetic_improvement.ollamachat import OllamaChat
from genetic_improvement.config import USER_PROMPT, UNIT_TEST_CODE, SYSTEM_PROMPT, NUM_VARIANTS, BSI_CLASSIFICATION
from genetic_improvement.genome import Genome

# CONSTS
POPULATION_SIZE = 8


In [2]:
# get the initial population from the LLM
population = []
for i in tqdm(range(POPULATION_SIZE // NUM_VARIANTS)):
    ollama_chat = OllamaChat(model="qwen3-coder:latest", system_prompt=SYSTEM_PROMPT, temperature=0.3)
    chat_response = ollama_chat.chat(user_text=USER_PROMPT, stream=False)

    # parse the chat response and add it to the population
    variants = OllamaChat.parse_variants(chat_response)
    candidates = OllamaChat.submit_variants(variants, classification=BSI_CLASSIFICATION)
    for candidate in candidates:
        if candidate is not None:
            genome = Genome(candidate_hash=candidate)
            population.append(genome)

 50%|█████     | 1/2 [00:42<00:42, 42.05s/it]

Code submitted successfully:  aaa5166111d2911e33a7c7c9f7d8adfc6ece33838b628dd588b63bb486cc64ac
Code submitted successfully:  716941edc374c94cc73973ea3f2f6b689de4a113beafce6eec83cf4caa5ccc3f
Code submitted successfully:  331556186cd1dd189d9d063a2ad6c5e471b6810267769f1a9235f374aa1a9594
Code submitted successfully:  4cb7ce589e97e20c02c30f5155114ac1804a1028861d8501fa60eca8a9df5730


100%|██████████| 2/2 [01:26<00:00, 43.12s/it]

Code submitted successfully:  a18cc44a77cc18c1fd0863c3f3b7f7e0b4711275da94c9fc08ebe9ec933a1007
Code submitted successfully:  f834a8a01bf6a1613276fbf4d1b5de76d188ccc904980dccf1af1a91f1610a9c
Code submitted successfully:  e93a42205d72c46cdc24a525ea2867d44528bffc29e0ff6bb170200d82fff9eb
Code submitted successfully:  a0d9bf2cf662a3556e59c2d408aa56528124a5cb44bed53cde13909ea2e4ad8a


In [3]:
# get the status of each candidate in the population
for genome in population:
    print(f"Candidate {genome.candidate_hash}")


Candidate aaa5166111d2911e33a7c7c9f7d8adfc6ece33838b628dd588b63bb486cc64ac
Candidate 716941edc374c94cc73973ea3f2f6b689de4a113beafce6eec83cf4caa5ccc3f
Candidate 331556186cd1dd189d9d063a2ad6c5e471b6810267769f1a9235f374aa1a9594
Candidate 4cb7ce589e97e20c02c30f5155114ac1804a1028861d8501fa60eca8a9df5730
Candidate a18cc44a77cc18c1fd0863c3f3b7f7e0b4711275da94c9fc08ebe9ec933a1007
Candidate f834a8a01bf6a1613276fbf4d1b5de76d188ccc904980dccf1af1a91f1610a9c
Candidate e93a42205d72c46cdc24a525ea2867d44528bffc29e0ff6bb170200d82fff9eb
Candidate a0d9bf2cf662a3556e59c2d408aa56528124a5cb44bed53cde13909ea2e4ad8a


In [4]:
# Database setup
import json

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from models import Sample, Tag, Analysis, Candidate

with open(os.path.join(project_root, "settings.json")) as f:
    settings = json.load(f)
DATABASE_URL = settings['sqlalchemy_database_uri']
engine = create_engine(DATABASE_URL, echo=False)
Session = sessionmaker(bind=engine)

In [7]:
for genome in population:
    print(genome.get_candidate())

Candidate ID: aaa5166111d2911e33a7c7c9f7d8adfc6ece33838b628dd588b63bb486cc64ac, Classification: com, Status: 2, F1: 0.25, F2: 0.3333333333333333, F3: None, Analysis ID: 17943, Error: None
Candidate ID: 716941edc374c94cc73973ea3f2f6b689de4a113beafce6eec83cf4caa5ccc3f, Classification: com, Status: 3, F1: 0.07692307692307693, F2: 0.0, F3: 0.0, Analysis ID: None, Error: Compilation failed: 4 errors, 0 warnings
Candidate ID: 331556186cd1dd189d9d063a2ad6c5e471b6810267769f1a9235f374aa1a9594, Classification: com, Status: 3, F1: 0.037037037037037035, F2: 0.0, F3: 0.0, Analysis ID: None, Error: Compilation failed: 2 errors, 20 warnings
Candidate ID: 4cb7ce589e97e20c02c30f5155114ac1804a1028861d8501fa60eca8a9df5730, Classification: com, Status: 3, F1: 0.05263157894736842, F2: 0.0, F3: 0.0, Analysis ID: None, Error: Compilation failed: 6 errors, 0 warnings
Candidate ID: a18cc44a77cc18c1fd0863c3f3b7f7e0b4711275da94c9fc08ebe9ec933a1007, Classification: com, Status: 2, F1: 0.25, F2: 0.3333333333333333